<a href="https://colab.research.google.com/github/VidhyaMadhavi/EVA/blob/master/Assignment15/Assignment15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment15

ASSIGNMENT 15
GROUP MEMBERS: 

vidhya.madhavi@gmail.com (vidhya),AYSWARYA, PAVAN, SMITA, YAMINI



### Work on the your own library now:
- Convert your code such that you have 4 additional files in 4 folders:
    data
    loss
    model
    optim_scheduler


Work on the your own library now:
Convert your code such that you have 4 additional files in 4 folders:
data
loss
model
optim_scheduler
Use these files to import related functions to train your model.
Your ipynb files can only have training loop, and everything else must be imported.
Look at this COMPLEX reference: https://github.com/rwightman/pytorch-image-models/tree/master/timm

In [0]:
# Upload to google drive and import

from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  gdrive  sample_data


In [0]:
import os
os.chdir('/content/drive/My Drive/EVA3-Assignment15')

In [0]:
!pwd

/content/gdrive/My Drive/EVA3-Assignment15


## Import required libaries, set parameters

In [0]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

from PIL import Image
import numpy as np
import skimage.io as io
from keras import backend as K

from models.model import init_pytorch, DavidNet




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.


## Enable Eager Mode

In [0]:

tf.enable_eager_execution()

**Set training parameters**

In [0]:


BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 24 #@param {type:"integer"}




##**Process the dataset and initialize the model**

In [0]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

# Defining the Optimization Strategies.

model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

def to_numpy(t):
  print("Inside to_numpy:", type(t.numpy()))
  t.numpy()

lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)
#data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(cutout(0.3, tf.py_function(to_numpy, [x], tf.float32)), [32, 32, 3])), y)


###**Loading the Dataset and converting to TfRecords**

### **Defining the Optimization Strategies**


## **Training the model**

Traing the dataset after decoding, and printing out the validation accuracy.

In [0]:

from data_1.data_1 import _bytes_feature, _convert_to_tfrecord, parse_record, generate_input_fn
train_tfrecords_filename = 'TrainCifar10.tfrecords'
test_tfrecords_filename = 'TestCifar10.tfrecords'


#Defining parameters

IMAGE_HEIGHT = 40
IMAGE_WIDTH = 40
IMAGE_DEPTH = 3
NUM_CLASSES = 10


train_tfrecords_filename = 'TrainCifar10.tfrecords'
test_tfrecords_filename = 'TestCifar10.tfrecords'

_convert_to_tfrecord(x_train, y_train,train_tfrecords_filename)
_convert_to_tfrecord(x_test,y_test,test_tfrecords_filename)


t = time.time()
test_set = generate_input_fn(test_tfrecords_filename, isTraining=False).batch(BATCH_SIZE)

hist_train_acc = []
hist_train_loss = []
hist_val_acc = []
hist_val_loss = []

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = generate_input_fn(train_tfrecords_filename).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
  tf.keras.backend.set_learning_phase(1)

  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    


  # Save data for plotting
  hist_train_acc.append(train_acc / len_train)
  hist_train_loss.append(train_loss / len_train)
  hist_val_acc.append(test_acc / len_test)
  hist_val_loss.append(test_loss / len_test)

  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

Generating TrainCifar10.tfrecords

Generating TestCifar10.tfrecords






epoch: 1 lr: 0.08 train loss: 1.5662189447021484 train acc: 0.43652 val loss: 1.4927370819091796 val acc: 0.4713 time: 48.46871781349182



epoch: 2 lr: 0.16 train loss: 0.8327390777587891 train acc: 0.7071 val loss: 1.2229104095458985 val acc: 0.6312 time: 85.15288925170898



epoch: 3 lr: 0.24 train loss: 0.6385257162475586 train acc: 0.7773 val loss: 0.8593535247802735 val acc: 0.7198 time: 122.29530382156372



epoch: 4 lr: 0.32 train loss: 0.5509262185668945 train acc: 0.81092 val loss: 0.7602496704101562 val acc: 0.7679 time: 159.20185089111328


...


epoch: 20 lr: 0.08421052631578951 train loss: 0.03724684064865112 train acc: 0.98922 val loss: 0.27969227981567385 val acc: 0.9233 time: 749.9816851615906



epoch: 21 lr: 0.06315789473684214 train loss: 0.031036935539245605 train acc: 0.99124 val loss: 0.25227938423156737 val acc: 0.9304 time: 786.7534584999084



epoch: 22 lr: 0.04210526315789476 train loss: 0.02761223198890686 train acc: 0.99264 val loss: 0.2521751739501953 val acc: 0.9286 time: 823.4677891731262



epoch: 23 lr: 0.02105263157894738 train loss: 0.023237625303268433 train acc: 0.99394 val loss: 0.24954226455688477 val acc: 0.9297 time: 860.4025523662567



epoch: 24 lr: 0.0 train loss: 0.020812588357925415 train acc: 0.99496 val loss: 0.2454846176147461 val acc: 0.9309 time: 897.2340667247772


### **Plotting training and validation accuracy and loss**

In [0]:
history = {}
history['acc'] = hist_train_acc
history['val_acc'] = hist_val_acc
history['loss'] = hist_train_loss
history['val_loss'] = hist_val_loss
import time
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline


def plot_model_history(history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(history['acc'])+1),history['acc'])
    axs[0].plot(range(1,len(history['val_acc'])+1),history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(history['acc'])+1),len(history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')

    # summarize history for loss
    axs[1].plot(range(1,len(history['loss'])+1),history['loss'])
    axs[1].plot(range(1,len(history['val_loss'])+1),history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(history['loss'])+1),len(history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()



plot_model_history(history)